## Import the libraries

In [3]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
import librosa.effects as effects



## Extract the Mfcc features and remove noise

In [4]:
from sklearn.model_selection import train_test_split

path = "/Users/mohamedabdallaoui/Desktop/Langue"
language = "russe"
genders = ["Femmes","Hommes"]

mfcc_folder_path = "/Users/mohamedabdallaoui/Desktop/Russe_MFCC"
mfcc_folder_path_femmes = os.path.join(mfcc_folder_path, "Femmes")
mfcc_folder_path_hommes = os.path.join(mfcc_folder_path, "Hommes")

mfcc_features_list = [] # To store MFCC features for all audio files

#This will loop through all the audio files in each gender, and load each audio file using librosa
for gender in genders:
    audio_files_path = os.path.join(path, language, gender)
    audio_files = os.listdir(audio_files_path)
    for audio_file in audio_files:
        audio_file_path = os.path.join(audio_files_path, audio_file)
        audio_signal, sample_rate = librosa.load(audio_file_path)

        #remove silence 
        audio_signal = effects.trim(audio_signal, top_db=40)[0]


        # Extract MFCC features
        mfcc_features = librosa.feature.mfcc(y=audio_signal, sr=sample_rate)
        mfcc_features_list.append(mfcc_features)
        
        # Save MFCC features in a new file in desktop
        if gender == "Hommes":
            for i in range(1, len(audio_files)+1):
                if f"h{i}.wav" in audio_file:
                    mfcc_file_path = os.path.join(mfcc_folder_path_hommes, f"{language}_{gender}_h{i}.npy")
                    np.save(mfcc_file_path, mfcc_features)

                    break
        elif gender == "Femmes":
            for i in range(1, len(audio_files)+1):
                if f"f{i}.wav" in audio_file:
                    mfcc_file_path = os.path.join(mfcc_folder_path_femmes, f"{language}_{gender}_f{i}.npy")
                    np.save(mfcc_file_path, mfcc_features)

                    break

# Pad MFCC features with zeros to make them of equal shape
# Maximum length of MFCC features
max_length = np.max([mfcc_features.shape[1] for mfcc_features in mfcc_features_list]) 
padded_features_list = []
for mfcc_features in mfcc_features_list:
    padded_features = librosa.util.pad_center(mfcc_features, size=max_length, axis=1)
    padded_features_list.append(padded_features)


## Split the data into train and test and save it in theire respective files

In [5]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Load all the MFCC files for a language and gender
language = "russe"
genders = ["Femmes", "Hommes"]
mfcc_folder_path = "/Users/mohamedabdallaoui/Desktop/Russe_MFCC"

mfcc_files = []
for gender in genders:
    audio_files_path = os.path.join(mfcc_folder_path, gender)
    audio_files = os.listdir(audio_files_path)
    for audio_file in audio_files:
        if audio_file.endswith(".npy"):
            mfcc_file_path = os.path.join(audio_files_path, audio_file)
            mfcc_files.append(mfcc_file_path)

# Split the MFCC files into train and test sets
train_files, test_files = train_test_split(mfcc_files, test_size=0.2, random_state=42)

# Create the train and test directories if they do not exist
train_dir = "/Users/mohamedabdallaoui/Desktop/train_russe"
test_dir = "/Users/mohamedabdallaoui/Desktop/test_russe"

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Save the train and test files in their respective directories

for file_path in train_files:
    if "russe_Femmes" in file_path:
        gender_folder = "Femmes"
    elif "russe_Hommes" in file_path:
        gender_folder = "Hommes"
    else:
        continue

    filename = os.path.basename(file_path)
    dest_path = os.path.join(train_dir, gender_folder, filename)
    np.save(dest_path, np.load(file_path))

for file_path in test_files:
    if "russe_Femmes" in file_path:
        gender_folder = "Femmes"
    elif "russe_Hommes" in file_path:
        gender_folder = "Hommes"
    else:
        continue

    filename = os.path.basename(file_path)
    dest_path = os.path.join(test_dir, gender_folder, filename)
    np.save(dest_path, np.load(file_path))


# Traing the model using Gmm

In [6]:
import os
import numpy as np
from sklearn.mixture import GaussianMixture

language = "russe"
genders = ["Femmes", "Hommes"]
train_dir = "/Users/mohamedabdallaoui/Desktop/train_russe"

# Load the train data
train_features = []
for gender in genders:
    gender_train_dir = os.path.join(train_dir, gender)
    audio_files = os.listdir(gender_train_dir)
    for audio_file in audio_files:
        if audio_file.endswith(".npy"):
            audio_file_path = os.path.join(gender_train_dir, audio_file)
            mfcc_features = np.load(audio_file_path)
            train_features.append(mfcc_features)

train_features = np.concatenate(train_features, axis=1)

In [30]:
# Fit the GMM on the train data
n_components =  64 #Number of mixture components
gmm = GaussianMixture(n_components=n_components, covariance_type="diag")
Model = gmm.fit(train_features.T)
Model

GaussianMixture(covariance_type='diag', n_components=64)

## Save model

In [32]:
from joblib import dump
# Save the trained model to a file
filename = '/Users/mohamedabdallaoui/Desktop/gmm_model.joblib'
dump(Model, filename)


['/Users/mohamedabdallaoui/Desktop/gmm_model.joblib']

# Predict the outcome

In [37]:
# Load the test data
test_dir = ""
test_dir = "/Users/mohamedabdallaoui/Desktop/langue/russe"
for language in os.listdir(test_dir):
    language_dir = os.path.join(test_dir, language)
    for gender in os.listdir(language_dir):
        gender_dir = os.path.join(language_dir, gender)
        audio_files = os.listdir(gender_dir)
        for audio_file in audio_files:
            if audio_file.endswith(".wav"):
                audio_file_path = os.path.join(gender_dir, audio_file)
                mfcc_features = extract_mfcc_features(audio_file_path)
                test_features.append(mfcc_features)

test_features = np.concatenate(test_features, axis=1)

# Predict the language
log_likelihoods = gmm.score_samples(test_features.T)[0]
language_idx = np.argmax(np.sum(log_likelihoods, axis=0))
predicted_language = os.listdir("/path/to/l'angue'")[language_idx]

print("Predicted language: ", predicted_language)


NotADirectoryError: [Errno 20] Not a directory: '/Users/mohamedabdallaoui/Desktop/langue/russe/.DS_Store'